In [2]:
using Distributions
using Plots
using PGFPlotsX
using LaTeXStrings
using LinearAlgebra
using Colors
using JLD
using Parameters
using Random
pgfplotsx()
push!(PGFPlotsX.CUSTOM_PREAMBLE, raw"\usepgfplotslibrary{fillbetween}");

In [ ]:
T = 80
n_steps = 50
gridman = LinRange(0, 1, n_steps)

### Heatmap example

In [3]:
@unpack efe_grids = JLD.load("jlds/efe_grids.jld")
efe_grids = reshape(efe_grids, n_steps,n_steps, T);
efe_grids[efe_grids .== Inf] .= maximum(efe_grids[efe_grids .!= Inf])

Dict{String, Any} with 1 entry:
  "efe_grids" => Any[22.3695, 22.452, 22.5289, 22.5996, 22.6638, 22.7213, 22.77…

In [78]:
index = 11
x = gridman
y = gridman

target = [[0.8], [0.2]]
current = map(x -> [first(x)], [0.5, 0.5])


axis1 = @pgf Axis(
    {
        xlabel=L"g_s",
        ylabel=L"g_n",
        zlabel="hz",
        legend_pos = "north east",
        legend_cell_align="{left}",
        grid = "major",
        view = (0, 90),
        colorbar,
        "colormap/hot",
        colorbar_style={"xlabel=EFE, xticklabel pos=upper"}
    },
    Plot({only_marks, color="green", mark_size=3.0, mark="*"}, 
    Coordinates(target...)), LegendEntry("Target"),
    Plot({only_marks, color="black", mark_size=3.0, mark="*"}, 
    Coordinates(current...)), LegendEntry("Current"),
    Plot3(
        {
            surf,
            shader = "flat",
        },
        Coordinates(x, y, efe_grids[:, :, index]))
    )

Axis(PGFPlotsX.Options(OrderedCollections.OrderedDict{Any, Any}("xlabel" => L"$g_s$", "ylabel" => L"$g_n$", "zlabel" => "hz", "legend_pos" => "north east", "legend_cell_align" => "{left}", "grid" => "major", "view" => (0, 90), "colorbar" => nothing, "colormap/hot" => nothing, "colorbar_style" => PGFPlotsX.Options(OrderedCollections.OrderedDict{Any, Any}("xlabel=EFE, xticklabel pos=upper" => nothing), true)…), true), Any[Plot(false, false, PGFPlotsX.Options(OrderedCollections.OrderedDict{Any, Any}("only_marks" => nothing, "color" => "green", "mark_size" => 3.0, "mark" => "*"), true), Coordinates{2}(Union{Nothing, Coordinate{2}}[Coordinate{2}((0.8, 0.2), nothing, nothing, nothing, nothing)]), Any[]), LegendEntry(PGFPlotsX.Options(OrderedCollections.OrderedDict{Any, Any}(), false), "Target", false), Plot(false, false, PGFPlotsX.Options(OrderedCollections.OrderedDict{Any, Any}("only_marks" => nothing, "color" => "black", "mark_size" => 3.0, "mark" => "*"), true), Coordinates{2}(Union{Nothi

In [79]:
pgfsave("tmp/heatmap_$(n).pdf", axis1)
pgfsave("tikz/hetamap_$(n).tikz", axis1)

### Time-series plots example

In [87]:
@unpack epi_grids = JLD.load("jlds/epi_grids.jld")
epi_grids = reshape(epi_grids, n_steps,n_steps, T);


In [88]:
@unpack inst_grids = JLD.load("jlds/inst_grids.jld")
inst_grids = reshape(inst_grids, n_steps,n_steps, T);

In [95]:
# EH?
idxs = [argmin(efe_grids[:, :, i]) for i in 1:T]

# Get the values of queried points for all time steps
epi_mins = [epi_grids[:,:,i][idxs[i]] for i in 1:T]
inst_mins = [inst_grids[:,:,i][idxs[i]] for i in 1:T]

# Get rid of infs for inhibition of return
epi_means = [mean(epi_grids[:,:,i][isfinite.(epi_grids[:,:,i])]) for i in 1:T]
inst_means = [mean(inst_grids[:,:,i][isfinite.(inst_grids[:,:,i])]) for i in 1:T]


In [100]:
n_samples = 100
index = 42

plt_agent = @pgf GroupPlot(
    # group plot options
    {
        width="15cm", height="8cm",

        group_style = {
            group_size = "2 by 1",
            horizontal_sep = "1.5cm",
        },
    },
    # first
    {xlabel=L"t",
           ylabel="value",
           legend_pos = "north east",
           legend_cell_align="{left}",
           grid = "major",
        #    title="TODO: TITLE",
           legend_style = "{nodes={scale=0.8, transform shape}}",
    },
    Plot(
        {no_marks,color="blue"}, Coordinates(collect(1:T), epi_mins - epi_means)
    ), LegendEntry("Epistemic value"),
    # second
    {xlabel=L"t",
           legend_pos = "south east",
           legend_cell_align="{left}",
           grid = "major",
        #    title="Dataset $(index)",
           legend_style = "{nodes={scale=0.8, transform shape}}",
    },
    Plot(
        {no_marks,color="blue"}, Coordinates(collect(1:T), inst_mins - inst_means)
    ), LegendEntry("Instrumental value"),
)

pgfsave("tmp/grouped_agent.pdf", plt_agent)
pgfsave("tikz/grouped_agent.tikz", plt_agent)